In [ ]:
def batcher(start, increment, total): 
    ''' Yields batch sizes for `total` items, with `start` index and `increment` batch size. \n
    Example 1: batcher(1, 20, 50) -> [20, 20, 9] \n
    Example 2: 
        for i, size in enumerate(batcher(1, 20, 50)):
            print(f"Batch {i+1} of size {size}: [{i*20+1}:{i*20+size}]")
    '''
    if start+increment > total:
        yield total-start
    else:
        yield increment
        yield from batcher(start+increment, increment, total)

print(list(batcher(1, 20, 50)))

In [ ]:
for i, size in enumerate(batcher(ds_skip_rows, batch_size, ds_rows_to_process)):
    start = ds_skip_rows + i*batch_size
    end = start + size
    print(f"Processing batch {i+1} of size {size}: [{start}:{end}]")
    batch = oaib.Auto(workers=8)
    for j, row in enumerate(ds.select(range(start, end))):
        messages=[{"role": "user", "content": row['prompt']}]
        await batch.add("chat.completions.create", 
                        model=model_name, 
                        messages=messages, 
                        max_tokens=normal_max_tokens_hard)                       
    output = await batch.run()
    ds = ds.map(with_indices=True, 
                function=lambda row, idx: {
                    'normal-answer': output.iloc[idx-start].result['choices'][0]['message']['content'] 
                        if start <= idx < end else None} )
    pprint(ds[4]['normal-answer'])

pprint(ds[4])

In [ ]:
# def calc_batch_sizes(total_rows, batch_size, skip_rows):
#     n_batches = math.ceil(total_rows / batch_size)
#     batch_sizes = []
#     start = skip_rows
#     for _ in range(n_batches):
#         end = min(start+batch_size, skip_rows+total_rows)
#         batch_sizes.append((end-start, start, end))
#         start = end
#     return batch_sizes

# # Example usage:
# batch_sizes = calc_batch_sizes(ds_rows_to_process, batch_size, ds_skip_rows)
# print(batch_sizes)


In [ ]:
# for b_i, (size, start, end) in enumerate(batch_sizes):
#     print(f"Processing batch {b_i+1} of {len(batch_sizes)}...")
#     batch_ds = ds.select(range(start, end))
#     batch = Auto(workers=8)
#     for j, row in enumerate(batch_ds):
#         # print(f"Batch {b_i}, row {j}, prompt: {row['prompt']}")
#         messages=[{"role": "user", "content": row['prompt']}]
#         await batch.add("chat.completions.create", model=model_name, 
#                         messages=messages, max_tokens=normal_max_tokens_hard)                       
#     output = await batch.run()
#     for row in output.itertuples():
#         print(f"{row.Index}: {batch_ds[row.Index]['question']}")
#         print(row.result["choices"][0]["message"]["content"])



In [ ]:

# append_dict = {
#     'question-id': ['001', '002'],
#     'question': ['What is the capital of France? \" \' 🤗 \' \"', 'What is the capital of Germany? </>'],
#     'brief-answer': ['Paris', 'Berlin'],
# }

# # Open the JSONL file in append mode with UTF-8 encoding
# # with open(out_file_name, 'ab') as file_handle:
# with open('data.jsonl', 'a', encoding='utf-8') as file_handle:
#     # Serialize the dictionary to a JSON-formatted string and write it to the file
#     for id, q, ba in zip(append_dict['question-id'], append_dict['question'], append_dict['brief-answer']):
#         entry = {
#             'question-id': id,
#             'question': q,
#             'brief-answer': ba,
#         }
#         # json.dump(append_dict, file_handle)
#         json.dump(entry, file_handle, ensure_ascii=False)
#         # Write a newline character to separate entries
#         file_handle.write('\n')
